Karman Filtering (Also known as Linear Quadratic Estimation (LQE)):

Structure:
- Produces estimates of current state variables
- Dynamic Positions of vehicles
- Useful as there is time delay between motor commands and sensory feedback, as the filtering allows for uncertainty.

Two-phase process:
- Prediction Phase
- Update Phase

In the prediction phase, the Kalman filter produces estimates of the current state variables, along with their uncertanties.
Once the outcome of the next measurement is observed, these estimates are updated using a weighted average, with more weight being given to estimates with greater uncertainty.


This can oeprate in real time, only using the present input measurements and the state calculated previously and its uncertainty matrix, no additional past info is required.

Optimal Karman filtering assumes that errors have a normal (gaussian) distribution.

In the words of Rudolf E. Kálmán: "In summary, the following assumptions are made about random processes: Physical random phenomena may be thought of as due to primary random sources exciting dynamic systems. The primary sources are assumed to be independent gaussian random processes with zero mean; the dynamic systems will be linear."

Though regardless of Gaussianity, if the process and measurement covariances are known, the Kalman filter is the best possible linear estimator in the minimum mean-square-error sense.

Extensions and generalizations of the method have also been developed, such as the extended Kalman filter and the unscented Kalman filter which work on nonlinear systems. The basis is a hidden Markov model such that the state space of the latent variables is continuous and all latent and observed variables have Gaussian distributions. Kalman filtering has been used successfully in multi-sensor fusion, and distributed sensor networks to develop distributed or consensus Kalman filtering.

Extended Kalman filtering (EKF) is used in robotics and pathfinding.

https://en.wikipedia.org/wiki/Kalman_filter

https://medium.com/@jaems33/understanding-kalman-filters-with-python-2310e87b8f48

SLAM (Simultaneous Localization And Mapping) notes:
https://www.cs.columbia.edu/~allen/F19/NOTES/slam_pka.pdf

Some solutions:
- Graph-SLAM

• Nodes represent poses or locations
• Constraints connect the poses of the
robot while it is moving
• Constraints are inherently uncertain 
• Observing previously seen areas
generates constraints between non-successive poses 




- EKF-SLAM
- Using Particle Filters (Rao-Blackwellized particle filter (FastSLAM))


https://www.doc.ic.ac.uk/~ajd/Robotics/RoboticsResources/SLAMTutorial2.pdf

Full comp sci course:
https://www.cs.columbia.edu/~allen/F19/notes.html


In [53]:
import numpy as np
from numpy.linalg import inv

Matrix_A = np.array([[1,0,0,0],[-1,1,0,0],[0,-1,1,0],[1,0,0,-1],[0,1,0,-1], [0,0,1,-1]])
Vector_X = []
Matrix_B = np.array([-3,5,3,-10,-5,-1])

ATA = np.matmul(Matrix_A.T, Matrix_A)
ATB = np.matmul(Matrix_A.T,Matrix_B)

#print(np.matmul(Matrix_A, Matrix_A.T))
Vector_X = np.matmul(np.linalg.inv(ATA), ATB)
print(Vector_X)

[-3.     2.125  5.5    6.875]


In [62]:
# With weights
import numpy as np
from numpy.linalg import inv

Matrix_A = np.array([[1,0,0,0],[-1,1,0,0],[0,-1,1,0],[1,0,0,-1],[0,1,0,-1], [0,0,1,-1]])
Vector_X = []
Matrix_B = np.array([-3,5,3,-10,-5,-1])

Weight_Matix = np.identity(6)
Weight_Matix[5,5] = 5 # Weight increased as X(2) should have less variance than the other positions.

ATWA = np.matmul(np.matmul(Matrix_A.T, Weight_Matix), Matrix_A)
ATWB = np.matmul(np.matmul(Matrix_A.T, Weight_Matix), Matrix_B)

#print(np.matmul(Matrix_A, Matrix_A.T))
Vector_X = np.matmul(np.linalg.inv(ATWA), ATWB)
print(Vector_X)

[[ 1 -1  0  1  0  0]
 [ 0  1 -1  0  1  0]
 [ 0  0  1  0  0  1]
 [ 0  0  0 -1 -1 -1]]
[-3.          2.17857143  5.71428571  6.82142857]


ATA = np.matmul(Matrix_A.T,Matrix_A)
ATB = np.matmul(Matrix_A.T,Matrix_B)
print(ATB)
print(Vector_X)

In [2]:
 
import numpy as np
from numpy.linalg import inv

x_observations = np.array([4000, 4260, 4550, 4860, 5110])
v_observations = np.array([280, 282, 285, 286, 290])

z = np.c_[x_observations, v_observations]

# Initial Conditions
a = 2  # Acceleration
v = 280
t = 1  # Difference in time

# Process / Estimation Errors
error_est_x = 20
error_est_v = 5

# Observation Errors
error_obs_x = 25  # Uncertainty in the measurement
error_obs_v = 6

def prediction2d(x, v, t, a):
    A = np.array([[1, t],
                  [0, 1]])
    X = np.array([[x],
                  [v]])
    B = np.array([[0.5 * t ** 2],
                  [t]])
    X_prime = A.dot(X) + B.dot(a)
    return X_prime


def covariance2d(sigma1, sigma2):
    cov1_2 = sigma1 * sigma2
    cov2_1 = sigma2 * sigma1
    cov_matrix = np.array([[sigma1 ** 2, cov1_2],
                           [cov2_1, sigma2 ** 2]])
    return np.diag(np.diag(cov_matrix))


# Initial Estimation Covariance Matrix
P = covariance2d(error_est_x, error_est_v)
A = np.array([[1, t],
              [0, 1]])

# Initial State Matrix
X = np.array([[z[0][0]],
              [v]])
n = len(z[0])

for data in z[1:]:
    X = prediction2d(X[0][0], X[1][0], t, a)
    # To simplify the problem, professor
    # set off-diagonal terms to 0.
    P = np.diag(np.diag(A.dot(P).dot(A.T)))

    # Calculating the Kalman Gain
    H = np.identity(n)
    R = covariance2d(error_obs_x, error_obs_v)
    S = H.dot(P).dot(H.T) + R
    K = P.dot(H).dot(inv(S))

    # Reshape the new data into the measurement space.
    Y = H.dot(data).reshape(n, -1)

    # Update the State Matrix
    # Combination of the predicted state, measured values, covariance matrix and Kalman Gain
    X = X + K.dot(Y - H.dot(X))

    # Update Process Covariance Matrix
    P = (np.identity(len(K)) - K.dot(H)).dot(P)

print("Kalman Filter State Matrix:\n", X)


[[4000  280]
 [4260  282]
 [4550  285]
 [4860  286]
 [5110  290]]
Kalman Filter State Matrix:
 [[5127.05898493]
 [ 288.55147059]]
